In [60]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import datetime
import re
import pickle
import math

import tensorflow as tf 
from tensorflow import keras
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.layers.experimental import preprocessing

In [3]:
data_filename = '/Users/brad/Desktop/CS6620/Project/Data/dataVersion1.npy' 
truth_filename = '/Users/brad/Desktop/CS6620/Project/Data/truthVersion1.npy' 
x = np.load(data_filename)
y = np.load(truth_filename)
print(x.shape)
print(y.shape)

(5259, 34)
(5259,)


In [6]:
x[0].shape

(34,)

In [9]:
train_length = int(x.shape[0] * 0.85)
x_train = x[0 : train_length].reshape(-1, 34, 1)
y_train = y[0 : train_length].reshape(-1, 1)
x_test = x[train_length : ].reshape(-1, 34, 1)
y_test = y[train_length : ].reshape(-1, 1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(4470, 34, 1)
(4470, 1)
(789, 34, 1)
(789, 1)


In [103]:
model = Sequential()
model.add(LSTM(16, input_shape=(34,1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))
# optimizer 
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-5)
# compile
model.compile(loss='mean_absolute_error', optimizer=opt)

In [95]:
def plot_loss(history):
  plt.figure()
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.grid(True)
  plt.show()

In [104]:
history= model.fit(x=x_train, y=y_train, batch_size=15, epochs=30, validation_data=(x_test, y_test))
plot_loss(history)

Epoch 1/30
298/298 [==============================] - 5s 13ms/step - loss: 0.1337 - val_loss: 0.0555
Epoch 2/30
298/298 [==============================] - 3s 10ms/step - loss: 0.1110 - val_loss: 0.0496
Epoch 3/30
298/298 [==============================] - 3s 10ms/step - loss: 0.1107 - val_loss: 0.0490
Epoch 4/30
298/298 [==============================] - 3s 10ms/step - loss: 0.1074 - val_loss: 0.0472
Epoch 5/30
298/298 [==============================] - 3s 11ms/step - loss: 0.1041 - val_loss: 0.0505
Epoch 6/30
298/298 [==============================] - 3s 9ms/step - loss: 0.1059 - val_loss: 0.0532
Epoch 7/30
298/298 [==============================] - 3s 10ms/step - loss: 0.1065 - val_loss: 0.0495
Epoch 8/30
298/298 [==============================] - 3s 9ms/step - loss: 0.1074 - val_loss: 0.0504
Epoch 9/30
298/298 [==============================] - 3s 10ms/step - loss: 0.1009 - val_loss: 0.0497
Epoch 10/30
298/298 [==============================] - 3s 9ms/step - loss: 0.1039 - val_loss:

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [105]:
preds = model.predict(x_test)

In [106]:
print(preds[40][0])
print(y_test[40][0])


0.057571024
0.030004727349974485


In [88]:
def computeAverageError(pred, y):
    err = []
    for i in range(len(pred)):
        err.append(abs((y[i] - pred[i][0])/y[i][0]))

    return sum(err)/ len(err)

In [107]:
computeAverageError(preds, y_test)

array([1.66981978])

In [108]:
preds2 = model.predict(x_train)
computeAverageError(preds2, y_train)[0]

1.450962010348354

In [101]:
plt.figure()
plt.scatter(x=np.arange(len(y_train)), y=y_train)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
plt.figure()
plt.scatter(x=np.arange(len(y_train)), y=preds2)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …